In [1]:
from utils import *

hete = Heterogeneity()
hete.check_torch_gpu()

-------------------------------------------------
------------------ VERSION INFO -----------------
Conda Environment: torchy | Python version: 3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]
Torch version: 2.0.1
Torch build with CUDA? True
# Device(s) available: 1, Name(s): Quadro P520



In [ ]:
X_data, y_data = np.load('X_data.npy'), np.load('y_data.npy')

xn = np.moveaxis(np.moveaxis(X_data, -2, 1).reshape(2000*61,64,64,4), -1, 1)
yn = np.moveaxis(np.moveaxis(y_data, -2, 1).reshape(2000*61,64,64,2), -1, 1)

In [6]:
# Hyperparameters and configurations
device = hete.device
print('device: {}'.format(device))

lr         = 2e-4
batch_size = 64
num_epochs = 10

input_channels_X  = 4
output_channels_X = 4
output_channels_Y = 2
input_channels_Y  = 2

device: cuda


In [12]:
train_dataset = NumpyDataset(xn, yn)
dataloader    = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [4]:
# Initialize CycleGAN
cycle_gan = CycleGAN(input_channels_X, output_channels_Y, input_channels_Y, output_channels_X).to(device)

# Loss functions
adversarial_loss       = nn.MSELoss()
cycle_consistency_loss = nn.L1Loss()

# Optimizers
optimizer_G   = Adam(cycle_gan.parameters(), lr=lr)
optimizer_D_X = Adam(cycle_gan.discriminator_X.parameters(), lr=lr)
optimizer_D_Y = Adam(cycle_gan.discriminator_Y.parameters(), lr=lr)

***
# END

In [7]:
for epoch in range(num_epochs):
    for i, (X, Y) in enumerate(dataloader):
        X = X.to(device)
        Y = Y.to(device)

        # Adversarial ground truths
        valid = torch.ones(X.size(0), 1, 14, 14).to(device)
        fake  = torch.zeros(X.size(0), 1, 14, 14).to(device)

        # ------------------
        #  Train Generators
        # ------------------

        optimizer_G.zero_grad()

        # Identity loss
        identity_X = cycle_gan.generator_YX(X)
        identity_Y = cycle_gan.generator_XY(Y)
        loss_identity = cycle_consistency_loss(identity_X, X) + cycle_consistency_loss(identity_Y, Y)

        # Adversarial loss
        fake_Y, fake_X, reconstructed_X, reconstructed_Y = cycle_gan(X, Y)
        loss_GAN_XY = adversarial_loss(cycle_gan.discriminator_Y(fake_Y), valid)
        loss_GAN_YX = adversarial_loss(cycle_gan.discriminator_X(fake_X), valid)
        loss_GAN = loss_GAN_XY + loss_GAN_YX

        # Cycle consistency loss
        loss_cycle_X = cycle_consistency_loss(reconstructed_X, X)
        loss_cycle_Y = cycle_consistency_loss(reconstructed_Y, Y)
        loss_cycle = loss_cycle_X + loss_cycle_Y

        # Total generator loss
        loss_G = loss_identity + loss_GAN + 10 * loss_cycle

        loss_G.backward()
        optimizer_G.step()

        # ----------------------
        #  Train Discriminators
        # ----------------------

        optimizer_D_X.zero_grad()

        # Real loss
        loss_real = adversarial_loss(cycle_gan.discriminator_X(X), valid)
        # Fake loss
        loss_fake = adversarial_loss(cycle_gan.discriminator_X(fake_X.detach()), fake)
        # Total discriminator X loss
        loss_D_X = (loss_real + loss_fake) / 2

        loss_D_X.backward()
        optimizer_D_X.step()

        optimizer_D_Y.zero_grad()

        # Real loss
        loss_real = adversarial_loss(cycle_gan.discriminator_Y(Y), valid)
        # Fake loss
        loss_fake = adversarial_loss(cycle_gan.discriminator_Y(fake_Y.detach()), fake)
        # Total discriminator Y loss
        loss_D_Y = (loss_real + loss_fake) / 2

        loss_D_Y.backward()
        optimizer_D_Y.step()

    # Print the losses for monitoring
    print('Epoch [{}/{}]: Generator Loss: {:.4f},  Discriminator Loss: {:.4f}'.format(epoch+1, num_epochs, 
                                                                              loss_G.item(), loss_D_X.item()+loss_D_Y.item()))
    
    # Save generated images (replace with your own path)
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            fake_Y, _, _, _ = cycle_gan(X, Y)
            fake_images = torch.cat((X, fake_Y), dim=0)
            save_image(fake_images, f"generated_images/epoch_{epoch + 1}.png", nrow=batch_size)

# Save the trained models (replace with your own path)
torch.save(cycle_gan.generator_XY.state_dict(), 'generator_XY.pth')
torch.save(cycle_gan.generator_YX.state_dict(), 'generator_YX.pth')
torch.save(cycle_gan.discriminator_X.state_dict(), 'discriminator_X.pth')
torch.save(cycle_gan.discriminator_Y.state_dict(), 'discriminator_Y.pth')


ValueError: too many values to unpack (expected 2)